In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from dotenv import load_dotenv

import os

e:\case\AI-hackaton-kemenkes\agent\ragagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
if not os.getenv("GEMINI_API_KEY") and not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GEMINI_API_KEY atau GOOGLE_API_KEY tidak ditemukan di environment variables.")

In [3]:
EMBEDDING_MODEL = GoogleGenerativeAIEmbeddings(model="text-embedding-004")
LLM_MODEL = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

In [4]:
class DataProcessor:
    def __init__(self, chunk_size: int = 1000, overlap: int = 200):
        self.textSplitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", ".", " "]
        )

    def processText(self, text: str) -> list[Document]:
        """ Memecah teks panjang menjadi chunks dan konversi ke objek Document """
        chunks = self.textSplitter.split_text(text)
        return [Document(page_content=chunk) for chunk in chunks]

In [5]:
class VectorDatabase:
    def __init__(self, embeddingFunction: GoogleGenerativeAIEmbeddings):
        self.embeddingFunction = embeddingFunction
        self.vectorStore = None
    
    def createIndex(self, documents: list[Document]):
        """ Membuat indeks vector store menggunakan FAISS dengan wrapper LangChain """

        self.vectorStore = FAISS.from_documents(
            documents=documents, 
            embedding=self.embeddingFunction
        )
        return self.vectorStore

In [6]:
class Retriever:
    def __init__(self, vectorStore: FAISS):
        self.retriever = vectorStore.as_retriever(search_kwargs={"k": 5})
    
    def retrieve(self, query: str) -> list[Document]:
        """ Melakukan pencarian pada vector store untuk menemukan data relevan """

        return self.retriever.invoke(query)

In [7]:
class ChatbotAgent:
    def __init__(self, vectorStore: FAISS, llm_model: ChatGoogleGenerativeAI):
        self.vectorStore = vectorStore
        self.llm = llm_model
        self.retriever = Retriever(vectorStore)
    
        # PROMPT RAG
        self.promptTemplate = ChatPromptTemplate.from_messages([
            ("system", """
                Anda adalah Asisten Pakar Gizi dan Pencegahan stunting untuk ibu hamil bernama MateBot panggil setiap user Bunda. Jawablah pertanyaan pengguna **HANYA** berdasarkan konteks yang diberikan di bawah.
                Pastikan jawaban Anda:
                1. Menggunakan bahasa Indonesia formal, ramah, dan informatif.
                2. Menyebutkan **Definisi**, **Penyebab**, dan **Pencegahan** jika relevan.
                3. Gunakan bullet point atau penomoran untuk memudahkan pembacaan.

                KONTEKS:
                {context}
                """),
                ("user", "{query}")
        ])

        self.FALLBACK_PROMPT = ChatPromptTemplate.from_messages([
            ("system", """
                Bunda bertanya tentang '{query}'. Informasi spesifik tidak ditemukan di basis data gizi MateBot.
                Jawab pertanyaan ini menggunakan pengetahuan umum Anda HANYA JIKA topik tersebut masih berhubungan erat dengan Gizi, Kehamilan, atau Stunting. 
                Jika pertanyaan sama sekali tidak berhubungan dengan topik ini (misalnya, sejarah, geografi, atau politik), jawab: 'Maaf Bunda, saya hanya dilatih untuk memberikan informasi spesifik mengenai gizi dan pencegahan stunting.' 
                Berikan jawaban dengan memanggil user 'Bunda' dan berikan disclaimer bahwa ini adalah informasi umum.
            """),
            ("user", "{query}")
        ])
    
    def generateResponse(self, query: str):
        """ Menghasilkan respons menggunakan retriever dan model generatif (Gemini) """
        
        DOC_COUNT = 5 
        DISTANCE_THRESHOLD = 0.4 

        scoredDocs = self.vectorStore.similarity_search_with_score(query, k=DOC_COUNT)
        
        bestDistance = scoredDocs[0][1] if scoredDocs else 999.0
        
        retrievedData = []
        
        if bestDistance > DISTANCE_THRESHOLD:
            response = self.llm.invoke(self.FALLBACK_PROMPT.format(query=query))
            retrievedData.append(Document(page_content="[SUMBER: Pengetahuan Umum MateBot (Tidak bersumber dari data gizi spesifik).]", metadata={"source": "Gemini Knowledge"}))
            
        else:
            retrievedData = self.retriever.retrieve(query)
            context = "\n---\n".join([doc.page_content for doc in retrievedData])
            
            response = self.llm.invoke(
                self.promptTemplate.format_messages(context=context, query=query)
            )

        return {
            "answer": response.content,
            "source_documents": retrievedData
        }

### ==============================================================================
### GLOBAL FUNCTIONS
### ==============================================================================

In [8]:
def processKnowledgeData(file_path: str = "../data/giziData.json"):
    """ Alur kerja Indexing: Load, Split, Embed, Store """
    loader = JSONLoader(
        file_path=file_path,
        jq_schema='.[]', 
        text_content=False, 
        content_key="content",
        
        metadata_func=lambda record: {
            "id": record.get("id", "N/A"),
            "topic": record.get("topic", "N/A"),
            "sub_topic": record.get("sub_topic", "N/A"),
            "source": record.get("source", "N/A"),
            "date_updated": record.get("date_updated", "N/A"),
        }
    )

    documents = loader.load()

    dataProcessor = DataProcessor() 
    documentsAfterSplit = dataProcessor.textSplitter.split_documents(documents)
    
    vectorDb = VectorDatabase(embeddingFunction=EMBEDDING_MODEL)
    vectorStore = vectorDb.createIndex(documentsAfterSplit)
    
    vectorStore.save_local(folder_path="./gizi_vector_index", index_name="gizi_index")
    
    return vectorStore

In [9]:
try:
    EMBEDDING_MODEL_RUNTIME = GoogleGenerativeAIEmbeddings(model="text-embedding-004")
    VECTOR_STORE_GLOBAL = FAISS.load_local(
        folder_path="../data/gizi_vector_index", 
        embeddings=EMBEDDING_MODEL_RUNTIME,
        index_name="gizi_index",
        allow_dangerous_deserialization=True
    )
    print("✅ Vector Store berhasil dimuat dari disk.")
except:
    print("⚠️ Vector Store tidak ditemukan, membuat indeks baru...")
    # VECTOR_STORE_GLOBAL = processKnowledgeData() 

✅ Vector Store berhasil dimuat dari disk.


In [10]:
def mainFlow(query):
    """ Fungsi utama untuk menjalankan RAG """
    # retriever = Retriever(VECTOR_STORE_GLOBAL) 
    chatbot = ChatbotAgent(
        vectorStore=VECTOR_STORE_GLOBAL, 
        llm_model=LLM_MODEL
    ) 
    response = chatbot.generateResponse(query) 
    return response


In [ ]:
query = "Apa itu Stunting?"
response = mainFlow(query)

print("--- Hasil Jawaban Chatbot ---")
print(response['answer'])

print("\n--- Dokumen Sumber yang Digunakan ---")
for doc in response['source_documents']:

    print(f"- {doc.page_content[:150]}...")